# Verificação do tempo de inferência para uma solução end-to-end

As dependências necessárias são importadas

In [4]:
import os

# Para adquirir o modelo pré-treinado
from fastai.vision.all import *

# Para iterar pelos vídeos e tornar interativo se desejado
import cv2

# Para extrair os rostos dos frames
from facenet_pytorch import MTCNN

# Útil para realizar operações em vetores
import numpy as np

# Dicionario para contabilizar os FAKE e os REAL
from collections import defaultdict

# Limpa prints extras nas células
from IPython.display import clear_output

Define-se o dispositivo que irá rodar as computações necessárias

In [5]:
# Definimos um device onde os tensores estarão sendo processados

device = torch.device('cpu')

In [6]:
# Informações para a MTCNN
IMAGE_SIZE = 224
MARGIN = 0
MIN_FACE_SIZE = 90
THRESHOLDS = [0.68, 0.75, 0.80]
POST_PROCESS = False
SELECT_LARGEST = True
KEEP_ALL = False
DEVICE = device

# ----------------------------------

mtcnn = MTCNN(image_size=IMAGE_SIZE,
              margin=MARGIN, 
              min_face_size=MIN_FACE_SIZE, 
              thresholds=THRESHOLDS,
              post_process=POST_PROCESS,
              select_largest=SELECT_LARGEST, 
              keep_all=KEEP_ALL, 
              device=device)

path_to_learner = Path('./models/final_learner.pkl')
learner = load_learner(path_to_learner, cpu=device.type == 'cpu') # As inferências nas imagens serão feitas pela CPU uma vez que será feito vídeo por vídeo

Define-se uma função que extrai os rostos dos vídeos utilizando a MTCNN e os guarda em lista lista.

In [17]:
def extract_faces_from_video(video_path, padding=0, resize_factor=0.6, check_every_frame=30, show_frames=False):
    
    # Captura o vídeo no path
    try:
        cap = cv2.VideoCapture(str(video_path))
    except:
        print("Ocorreu um erro ao carregar o vídeo. Certifique-se de que é um arquivo de vídeo válido.")
        return []
    
    # Pega, em inteiros, a quantidade de frames do vídeo
    v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = cap.get(cv2.CAP_PROP_FPS)

    faces = []
    for i in range(1, v_len + 1):
        success = cap.grab()
        # Apertar a tecla 'q' para sair do vídeo.
        if not success:
            continue
        if  i % check_every_frame == 0:
            success, frame = cap.retrieve()
            if not success:
                continue
        else:
            continue
        
        if success: # Sucesso na leitura
            # Obtém o frame como PIL Image (ele é capturado no formato BGR porém a MTCNN espera no formato RGB)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb = Image.fromarray(frame_rgb)
            if show_frames:
                frame = cv2.resize(frame, (int(frame.shape[1]*resize_factor), int(frame.shape[0]*resize_factor)))
                cv2.imshow('frame', frame)

            boxes, _ = mtcnn.detect(frame_rgb) # Detecta as imagens. O método detect só aceita numpy arrays
            
            if boxes is not None: # Só entra se rostos forem detectados
                for box in boxes: # Para cada uma das bouding boxes encontradas em um único frame (a princípio só deve ter uma)
                    box = [int(b) for b in box]
                    # Extrai a face
                    face = frame_rgb.crop(box=(box[0]-padding, 
                                               box[1]-padding, 
                                               box[2]+padding, 
                                               box[3]+padding))
                    faces.append(PILImage(face))

                    if show_frames:
                        frame = cv2.resize(frame, (int(frame.shape[1]*resize_factor), int(frame.shape[0]*resize_factor)))
                        cv2.imshow('frame', frame)
                        if cv2.waitKey(1) & 0xFF == ord('q'):
                            break
                    
                
        else:
            continue
    
    cv2.destroyAllWindows()
    cap.release()
    
    return faces

In [8]:
def get_predictions(learner, faces, verbose=False):
    predicts = []
    predicts_dict = defaultdict(lambda: 0)
    for i, face in enumerate(faces):
        res = learner.predict(face)
        clear_output()
        if verbose:
            print(f"Predição realizada para a face {i+1}")
        predicts.append(res[1].item())
        predicts_dict[res[0]] += 1
    print('-'*100)
    print(F"Resultados individuais: Quantidade de \033[91m FAKES \033[0m: {predicts_dict['FAKE']} | Quantidade de \033[92m REALS \033[0m: {predicts_dict['REAL']}")
    return predicts

In [9]:
def get_final_prediction_from_predictions(predictions, roh=2.75):
    print("-"*100)
    print(f"Utilizando regra com ρ = {roh}...\n")
    if not isinstance(predictions, np.ndarray):
        predictions = np.array(predictions)
    qtd_fakes = np.count_nonzero(predictions == 0)
    qtd_reals = np.count_nonzero(predictions == 1)
    
    return 'FAKE' if qtd_fakes >= roh*qtd_reals else 'REAL'

In [51]:
folder = "dfdc_train_part_0"
list_of_paths = list(Path(f"./Kaggle Dataset/{folder}").glob("*.mp4"))[:10] # 10 primeiros vídeos
list_of_paths[:5]

[Path('Kaggle Dataset/dfdc_train_part_0/aaqaifqrwn.mp4'),
 Path('Kaggle Dataset/dfdc_train_part_0/aayrffkzxn.mp4'),
 Path('Kaggle Dataset/dfdc_train_part_0/abhggqdift.mp4'),
 Path('Kaggle Dataset/dfdc_train_part_0/acagallncj.mp4'),
 Path('Kaggle Dataset/dfdc_train_part_0/acdkfksyev.mp4')]

In [52]:
check_every_frame=30
rho=2.75

tempo_corrido = []
qtde_rostos = []
for path in list_of_paths:
    inicio = time.time()
    
    faces = extract_faces_from_video(path, check_every_frame=check_every_frame)
    if len(faces) == 0:
        print("Não foi possível detectar faces humanas no vídeo fornecido.")
    else:
        preds = get_predictions(learner, faces)
        final_res = get_final_prediction_from_predictions(preds, rho)
    
    final = time.time()

    tempo_corrido.append((final-inicio)) # Pegamos o tempo total e dividimos pela quantidade de faces
    qtde_rostos.append(len(faces))

----------------------------------------------------------------------------------------------------
Resultados individuais: Quantidade de  FAKES : 10 | Quantidade de  REALS : 0
----------------------------------------------------------------------------------------------------
Utilizando regra com ρ = 2.75...



In [53]:
tempo_corrido = np.array(tempo_corrido)
qtde_rostos = np.array(qtde_rostos)

In [61]:
tempo_corrido, qtde_rostos, tempo_corrido*1000/qtde_rostos

(array([2.82597709, 2.96306109, 2.85306478, 2.73500085, 2.84399891,
        2.9134295 , 2.87600112, 3.06052232, 2.88699961, 2.77999902]),
 array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10]),
 array([282.5977087 , 296.30610943, 285.30647755, 273.50008488,
        284.3998909 , 291.34294987, 287.60011196, 306.05223179,
        288.69996071, 277.99990177]))

In [62]:
tempo_corrido.mean(), tempo_corrido.std()

(2.8738054275512694, 0.08732196321761623)